In [1]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from dotenv import load_dotenv
import os
import datetime

c:\Users\yq198\Desktop\DANNY_AI\AI_REVISED\CV_JD_AI\venv\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\yq198\Desktop\DANNY_AI\AI_REVISED\CV_JD_AI\venv\Lib\site-packages\pydantic\_internal\_fields.py:198: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
c:\Users\yq198\Desktop\DANNY_AI\AI_REVISED\CV_JD_AI\venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:617: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this err

In [ ]:
# Load environment variables
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [ ]:
# Initialize LLM and Tool
llm = "gpt-4-0125-preview"
search_tool = SerperDevTool()

companies = input("Enter companies to analyze (comma-separated, e.g., Apple, Tesla): ").split(", ")
companies_str = ", ".join(companies)

# Get the current date and time
current_time = datetime.datetime.now()
filename_datetime = current_time.strftime("%Y/%m/%d/%H")

In [ ]:
# Define Agents with company-specific focus and search tool
research_agent = Agent(
    role='Company Researcher',
    goal=f'Gather financial data and market position for {companies_str} up to the latest available date and time, specifically {current_time}.',
    backstory='You are an expert in company-specific financial research with access to latest available date and time web data.',
    verbose=True,
    llm=llm,
    tools=[search_tool]
)

analysis_agent = Agent(
    role='Company Analyst',
    goal=f'Analyze investment potential for {companies_str} based on the latest data available up to {current_time}.',
    backstory='You specialize in evaluating company investment opportunities.',
    verbose=True,
    llm=llm,
    tools=[search_tool]
)

report_agent = Agent(
    role='Investment Advisor',
    goal=f'Create investment recommendations for {companies_str} with current insights up to {current_time}.',
    backstory='You are a financial advisor creating tailored investment reports.',
    verbose=True,
    llm=llm
)

In [ ]:
# Define Tasks
research_task = Task(
    description=f'Research current financial performance, market trends, and news for {companies_str} up to {current_time} using web search.',
    agent=research_agent,
    expected_output=f'A summary of financial data and market position for {companies_str} based on data up to {current_time}.'
)

analysis_task = Task(
    description=f'Analyze the investment potential of {companies_str} including risk assessment using current data up to {current_time}.',
    agent=analysis_agent,
    expected_output=f'Detailed investment analysis for {companies_str} with risk levels based on trends up to {current_time}.'
)

report_task = Task(
    description=f'Create an investment recommendation report for {companies_str} based on the latest analysis up to {current_time}.',
    agent=report_agent,
    expected_output=f'A structured report with investment recommendations for {companies_str} based on data up to {current_time}.'
)

In [ ]:
# Create and Run the Crew
investment_crew = Crew(
    agents=[research_agent, analysis_agent, report_agent],
    tasks=[research_task, analysis_task, report_task],
    verbose=True
)

In [ ]:
# Execute the crew
result = investment_crew.kickoff()

# Display and Save Results
print(f"Investment Report for {companies_str} ({filename_datetime}):")
print(result)

# Save with company-specific filename
filename = f"investment_report_{'_'.join(companies)}_{filename_datetime.replace('/', '_')}.txt"
with open(filename, 'w') as f:
    f.write(str(result))
print(f"Report saved to '{filename}'")